In [1]:
import pandas as pd
import os
import random
from tqdm import tqdm

def process_and_save_direct(df, output_dir='./data_test', val_ratio=0.05):
    """
    데이터프레임을 받아 Phase를 추가하고, Episode 단위로 Train/Val을 나누어 바로 저장합니다.
    저장 경로: 
      - ./data/train/{game_id}_{episode_id}.csv
      - ./data/val/{game_id}_{episode_id}.csv
    """
    
    # 1. Phase 컬럼 추가 (전체 데이터에서 한 번에 연산하는 것이 빠름)
    # team_id가 변하는 순간을 새로운 phase의 시작으로 간주
    if 'team_id' not in df.columns:
        raise ValueError("데이터에 'team_id' 컬럼이 필요합니다.")
        
    print("Phase 정보 생성 중...")
    # shift(1)과 비교하여 팀이 바뀌면 True(1), 아니면 False(0) -> 누적합(cumsum)으로 ID 생성
    df['phase'] = (df['team_id'] != df['team_id'].shift(1)).fillna(0).cumsum()

    # 2. 저장 폴더 생성
    train_dir = os.path.join(output_dir, 'train')
    val_dir = os.path.join(output_dir, 'val')
    
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(val_dir, exist_ok=True)

    # 3. Episode 단위로 그룹화
    # (game_id, episode_id) 쌍을 키로 사용하여 그룹핑
    print("에피소드 그룹화 중...")
    grouped = df.groupby(['game_id', 'episode_id'])
    
    # 모든 에피소드의 키 리스트 추출 [(game_id, ep_id), ...]
    episode_keys = list(grouped.groups.keys())
    
    # 4. 랜덤 셔플 및 분할 (Episode 단위)
    random.seed(42) # 재현성을 위해 시드 고정
    random.shuffle(episode_keys)
    
    total_episodes = len(episode_keys)
    val_count = int(total_episodes * val_ratio)
    
    # Validation으로 사용할 키들을 Set으로 만들어 검색 속도 향상
    val_keys_set = set(episode_keys[:val_count])
    
    print(f"Total Episodes: {total_episodes}")
    print(f"Train: {total_episodes - val_count}, Val: {val_count}")

    # 5. 분할하여 저장
    for key in tqdm(episode_keys, desc="Saving episodes"):
        game_id, episode_id = key
        
        # 해당 그룹의 데이터 가져오기
        episode_df = grouped.get_group(key)
        
        # 파일명 생성
        file_name = f"{game_id}_{episode_id}.csv"
        
        if key in val_keys_set:
            save_path = os.path.join(val_dir, file_name)
        else:
            save_path = os.path.join(train_dir, file_name)
            
        # CSV 저장
        episode_df.to_csv(save_path, index=False)

    print("모든 데이터 처리가 완료되었습니다.")

# --- 사용 예시 ---
if __name__ == "__main__":
    # 1. 데이터 로드 (가지고 계신 데이터 파일 경로로 수정해주세요)
    df = pd.read_csv('./open_track1/train.csv')
    
    # 2. 함수 실행
    process_and_save_direct(df)

Phase 정보 생성 중...
에피소드 그룹화 중...
Total Episodes: 15435
Train: 14664, Val: 771


Saving episodes: 100%|██████████| 15435/15435 [01:16<00:00, 201.43it/s]

모든 데이터 처리가 완료되었습니다.
